## Setting up

In [2]:
import os
os.chdir("/Users/mingyu/Desktop/ICA")

In [1]:
from global_settings import DATA_PATH
from scipy.spatial import procrustes
from global_settings import PATH_DICT
import pickle5 as pickle
import numpy as np
from torch import nn
import torch
import re

In [2]:
def get_fitted(activation, model_name, print_disp=False):
    with open(os.path.join(DATA_PATH, f"params_{m}_{n}.pkl"), "rb") as handle:
        params = pickle.load(handle)
        w, b = params["w"], params["b"]
    
    activation_name = ''.join([_ for _ in re.sub("[\(\[].*?[\)\]]", "", str(activation)) if _.isalpha()])
    model_path = os.path.join(PATH_DICT[model_name], f"m{m}_n{n}_{activation_name}")
    model = torch.load(os.path.join(model_path, "model.pth"))
    w_hat = model["decoder.fc.weight"].numpy().astype(np.float64)
    b_hat = model["decoder.fc.bias"].numpy().astype(np.float64)
    
    if print_disp:
        # print disparity 
        b = b.reshape(len(b), 1)
        b_hat = b_hat.reshape(len(b_hat), 1)
        w_disp = procrustes(w, w_hat)[2]
        b_disp = procrustes(b, b_hat)[2]
        print(f"Disparity for weight matrix: {round(w_disp, 3)}")
        print(f"Disparity for bias vector: {round(b_disp, 3)}")
    
    return w_hat, b_hat

In [3]:
m, n = 2, 2

### 0. Ground Truth

In [4]:
with open(os.path.join(DATA_PATH, f"params_{m}_{n}.pkl"), "rb") as handle:
    params = pickle.load(handle)
    w, b = params["w"], params["b"]

In [5]:
w

array([[2.31396193, 0.06225585],
       [1.9009447 , 2.24641165]])

In [6]:
b

array([0.49850701, 0.22479665])

### 1.1 ReLU with MLE

In [7]:
model_name = "mleauto"
activation = nn.ReLU()
get_fitted(activation, model_name)

(array([[-2.03450799,  1.16307735],
        [-0.56623292,  2.84901834]]),
 array([0.4938812, 0.249731 ]))

### 1.2 ReLU with VAE

In [8]:
model_name = "vae"
activation = nn.ReLU()
get_fitted(activation, model_name)

(array([[-0.16225757, -1.74038982],
        [-0.412806  , -2.22019005]]),
 array([1.04388797, 0.96761191]))

### 2.1 Sigmoid with MLE

In [9]:
model_name = "mleauto"
activation = nn.Sigmoid()
get_fitted(activation, model_name)

(array([[ 2.03217411, -1.70263267],
        [ 0.5922696 , -1.80718672]]),
 array([0.56725401, 0.21412075]))

### 2.2 Sigmoid with VAE

In [10]:
model_name = "vae"
activation = nn.Sigmoid()
get_fitted(activation, model_name)

(array([[-2.2580688 ,  4.6960187 ],
        [-4.70776415, -1.98049355]]),
 array([ 0.77754241, -0.092829  ]))

### 3.1 Tanh with MLE

In [11]:
model_name = "mleauto"
activation = nn.Tanh()
get_fitted(activation, model_name)

(array([[2.42021847, 0.71098846],
        [1.03510618, 2.25148892]]),
 array([0.53990328, 0.21895871]))

### 3.2 Sigmoid with VAE

In [14]:
model_name = "vae"
activation = nn.Tanh()
get_fitted(activation, model_name)

(array([[ 0.00237525, -1.38529801],
        [-0.00328783, -1.41264927]]),
 array([0.6198566 , 0.44329619]))

### 4.1 LeakyReLU with MLE

In [13]:
model_name = "mleauto"
activation = nn.LeakyReLU()
get_fitted(activation, model_name)

(array([[2.27939367, 0.56431121],
        [1.32160783, 2.58895802]]),
 array([0.49619189, 0.25369683]))

### 4.2 LeakyReLU with VAE

In [15]:
model_name = "vae"
activation = nn.LeakyReLU()
get_fitted(activation, model_name)

(array([[-0.91012222, -0.0071652 ],
        [-1.2368201 ,  0.0081029 ]]),
 array([0.49280033, 0.19474944]))